# Email Demo

# Part 0: Imports

In [1]:
from file_utils import *
# from galeShapleyAlgorithmQuota import *
from match import *
from tqdm import tqdm # For a nice interface

## Part 1: Email Applications

**Note: It is recommended to use a quota of something like 1,2,1.**

In [10]:
P1, df_filtered, df_original = load_csv("example_preferences.csv")

# print()
course_dfs = courseTopChoices(df_original)

ids: ['A1' 'A2' 'A3' 'A4' 'A5' 'A6']
matrix before: [[4 2 1]
 [5 2 5]
 [1 4 2]
 [2 1 4]
 [1 5 2]
 [5 2 1]]
matrix after: [['A1' 4 2 1]
 ['A2' 5 2 5]
 ['A3' 1 4 2]
 ['A4' 2 1 4]
 ['A5' 1 5 2]
 ['A6' 5 2 1]]
ULA Preferences Requested for {{current_quarter}}


3it [00:00, 22429.43it/s]


['5A (Solis S W).csv']
['5B (Solis S W).csv']
['9 (Wang R K).csv']
No test email will be sent.

----------NAME AND EMAILS----------
NAME                , EMAIL
Solis S W           , erikfeng+solis_s_w@ucsb.edu
Solis S W           , erikfeng+solis_s_w@ucsb.edu
Wang R K            , erikfeng+wang_r_k@ucsb.edu

----------VARIABLES----------
current_quarter: Winter 25
next_quarter: Spring 25

----------SUBJECT----------
ULA Preferences Requested for Winter 25

----------MESSAGE(HTML)----------
<p>Dear [[professor]],</p>

<p>Please fill out your preferences in the attached CSV file. Use the empty column to indicate your preferences, where a <strong>lower</strong> number indicates a <strong>higher</strong> preference. </p>

<p>You have been allocated a quota of [[quota]] LA(s). Please indicate preferences of approximately 2-3 times the number of your quota, and provide a completed CSV file with your preferences.</p>

<p>Regards,<br />
Ziad Matni &amp; Phill Conrad, ULA Program Chairs</p>

<p

100%|██████████| 3/3 [00:04<00:00,  1.42s/it]

All emails sent.


### Part 1.5: Example Professor Rankings
An example of how professors might rank students (this is done randomly).

Professors rank about 2 times the quota of students they're given, ±15% of their quota or 1 student, with a 20% possibility per student being a tie. Professors always rank at least 1 student.



In [2]:
from pathlib import Path
import re
import pandas as pd
import random
import numpy as np




def exampleProfRank(course_dfs: tuple[str, Path], template: str = "templates/professor_preferences.csv", folder = Path("completed_preferences")) -> None:
    for name, df in course_dfs:
        df.to_csv(f'{folder}/{name}', index=False)
        

    quota = iter(pd.read_csv(template)["quota"].tolist())

    for csv_path in sorted(folder.glob("*.csv")):
        df = pd.read_csv(csv_path)
        q = 2*next(quota)
        n = len(df)  # number of data rows (header excluded)
        if n == 0:
            continue

        # ---- compute r per your rules ----
        p = max(0.15 * q, 1)
        lo, hi = q - p, q + p
        if lo > n or hi > n:
            lo = max(lo, n)
            hi = min(hi, n)
        lo = max(lo, 1)
        # if lo >= hi: r = q
        r = int(round(random.uniform(lo, hi)))

        if r >= n: r = n

        # ---- pick r random rows to receive the sequence ----
        idx = np.random.choice(n, size=r, replace=False)

        # ---- build sequence with 20% chance of "tie" (no increment) ----
        # Have to do this weird thing to clear cells without deprecation warnings
        col0 = df.columns[0]
        df[col0] = pd.array([""] * len(df), dtype=str)  # NEW: clear + set dtype


        
        if r > 0:
            vals = [1]
            for _ in range(1, r):
                vals.append(vals[-1] if random.random() < 0.20 else vals[-1] + 1)
            # assign into first column; header is untouched
            df.iloc[idx, 0] = vals

        df.to_csv(csv_path, index=False)


In [12]:
exampleProfRank(course_dfs=course_dfs)

### Part 2: Matching Preferences
With all the data, we can now match the preferences.

### Part 2.1: Recombining Professor Preferences

In [13]:
recombine_preferences()

### Part 2.2: Reading Professor and Student Preferences, Quotas

In [3]:
P1, df_filtered, df_original = load_csv("example_preferences.csv")
P2, df_prof = load_prof("prof_preferences.csv")
quotas = getQuotas()

ids: ['A1' 'A2' 'A3' 'A4' 'A5' 'A6']
matrix before: [[4 2 1]
 [5 2 5]
 [1 4 2]
 [2 1 4]
 [1 5 2]
 [5 2 1]]
matrix after: [['A1' 4 2 1]
 ['A2' 5 2 5]
 ['A3' 1 4 2]
 ['A4' 2 1 4]
 ['A5' 1 5 2]
 ['A6' 5 2 1]]


### Part 2.3: Matching Professor and Student Preferences

In [4]:
match_dict = match(P1, P2, quotas, df_original, df_filtered, print_statistics=True)

100%|██████████| 10000/10000 [00:00<00:00, 18724.49it/s]


avg incorrect: 55948
avg avg total diff: 5.5948
avg avg diff: 1.11896


ALL PAIRED MATCHES:
A1 5A
A2 9
A4 5B
A5 5B


{'5A': ['A1'], '9': ['A2'], '5B': ['A4', 'A5']}
{'5B': [0, 1], '5A': [0, 1], '9': [0, 1]}
Skipped matches: {}
class_exp_dict: dict_values(['No inexperienced', 'No inexperienced', 'No inexperienced'])
the whole dictionary {'5A': 'No inexperienced', '9': 'No inexperienced', '5B': 'No inexperienced'}
Proportion of mixed exp classes: 0.0%


### Part 2.4: Sending out acceptance Emails

Emails are handled via Jennifer Torres, so no email needs to be sent?

In [18]:
createAcceptanceTracker("example_preferences.csv", match_dict) # This does not do anything besides create a CSV of all students

'student_responses.csv'

### Part 2.5: Sending out rejection Emails

In [19]:
sendRejectionEmails()

                  Email Address First name Last name  ID assignment  \
0  erikfeng+Adam1Smith@ucsb.edu      Adam1     Smith  A1          9   
1  erikfeng+Adam2Smith@ucsb.edu      Adam2     Smith  A2         5A   
2  erikfeng+Adam3Smith@ucsb.edu      Adam3     Smith  A3        NaN   
3  erikfeng+Adam4Smith@ucsb.edu      Adam4     Smith  A4         5B   
4  erikfeng+Adam5Smith@ucsb.edu      Adam5     Smith  A5         5B   
5  erikfeng+Adam6Smith@ucsb.edu      Adam6     Smith  A6        NaN   

         status  
0  pre-notified  
1  pre-notified  
2   unconfirmed  
3  pre-notified  
4  pre-notified  
5   unconfirmed  
                  Email Address First name Last name  ID assignment  \
2  erikfeng+Adam3Smith@ucsb.edu      Adam3     Smith  A3        NaN   
5  erikfeng+Adam6Smith@ucsb.edu      Adam6     Smith  A6        NaN   

        status  
2  unconfirmed  
5  unconfirmed  
LA positions for {{current_quarter}} have been filled



2it [00:00, 17848.10it/s]


[]
[]
No test email will be sent.

----------NAME AND EMAILS----------
NAME                , EMAIL
erikfeng+Adam3Smith@ucsb.edu, Adam3
erikfeng+Adam6Smith@ucsb.edu, Adam6

----------VARIABLES----------
current_quarter: Winter 25
next_quarter: Spring 25

----------SUBJECT----------
LA positions for Winter 25 have been filled

----------MESSAGE(HTML)----------
<p>Dear [[applicant]],</p>

<p>Thank you for applying to be a Learning Assistant for the CS department for Winter 25. We had many applicants, and it was difficult to choose among them for the limited slots in the program. Thank you for your patience. <strong>Unfortunately, you have not been selected for a ULA position in the Winter 25 quarter.</strong></p>

<p>We got many more applications as we have available positions, but please do not be discouraged as it sometimes takes a couple of application cycles to get picked up for the first time, and unfortunately, this sometimes happens to experienced ULAs as well, even though once you

100%|██████████| 2/2 [00:02<00:00,  1.04s/it]

All emails sent.


## Part 2: Parse the dummy file

In [ ]:
import csv

## Part 2: Run the script

In [ ]:
from mail_utils import send_emails

In [ ]:
send_emails("ula_rejection")